In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install gdown
import gdown

# url = 'https://drive.google.com/file/d/1ebZupCn9BiwfQ-a4Rn3CA78dYQHs25DW/view?usp=sharing'
# output = 'dataset.zip'
# gdown.download(url, output, quiet=False)
# !gdown https://drive.google.com/uc?id=1ebZupCn9BiwfQ-a4Rn3CA78dYQHs25DW
!gdown https://drive.google.com/uc?id=12wtUMyGmlyB8C99QODZtNxxSLs_hW6ME

Mounted at /content/drive
Downloading...
From (original): https://drive.google.com/uc?id=12wtUMyGmlyB8C99QODZtNxxSLs_hW6ME
From (redirected): https://drive.google.com/uc?id=12wtUMyGmlyB8C99QODZtNxxSLs_hW6ME&confirm=t&uuid=ae7e7ffe-f700-4566-ad4b-16b9c676df34
To: /content/dataset.zip
100% 2.64G/2.64G [00:47<00:00, 55.7MB/s]


In [3]:
!unzip -q dataset.zip -d dataset/

In [4]:
# !gdown https://drive.google.com/uc?id=1wmNAUjJcDD6kptRMncHCflkmw5nlTjFe
!gdown https://drive.google.com/uc?id=1WBLyO_J06lyvqg1wAkb1rPVxi-5yt8yF

Downloading...
From: https://drive.google.com/uc?id=1WBLyO_J06lyvqg1wAkb1rPVxi-5yt8yF
To: /content/meta.json
100% 104k/104k [00:00<00:00, 119MB/s]


In [5]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=e2d048586d96e66e22f7397937fd133ee89d4fa008fa5626a5bb67c7639b1a46
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
# from torchvision.models import EfficientNet
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch.optim as optim
#from datasets import data_generator,collect_all_labels
from sklearn.preprocessing import MultiLabelBinarizer
##from my_datasets import MyDataset
import numpy as np
from PIL import Image

import os
import json
from sklearn.preprocessing import MultiLabelBinarizer
#from datasets import data_generator,collect_all_labels
import torch
import cv2 as cv
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau


def data_generator(img_dir, annotation_dir, batch_size, mlb):
    img_paths = os.listdir(img_dir)
    img_paths = [img for img in img_paths if img.endswith((".jpg", ".jpeg", ".png"))]

    while True:
        # Shuffle images each epoch
        np.random.shuffle(img_paths)

        for start in range(0, len(img_paths), batch_size):
            end = min(start + batch_size, len(img_paths))
            batch_img_paths = img_paths[start:end]

            batch_labels = []
            imgs = []

            for img_name in batch_img_paths:
                img_path = os.path.join(img_dir, img_name)
                # img = load_img(img_path, target_size=(240, 240))  # Ensure images are resized
                # img = img_to_array(img)
                img=cv.imread(img_path)
                img=cv.resize(img,(240,240))
                imgs.append(img)

                annotation_path = os.path.join(annotation_dir, img_name + '.json')
                with open(annotation_path, 'r') as f:
                    annotation = json.load(f)
                img_labels = [obj["classTitle"] for obj in annotation.get('objects', [])]
                batch_labels.append(set(img_labels))

            # Use the pre-fitted MultiLabelBinarizer to transform labels into binary arrays
            batch_labels = mlb.transform(batch_labels)

            imgs = np.array(imgs, dtype="float32") / 255.0  # Normalize images

            yield imgs, np.array(batch_labels)

def collect_all_labels(annotation_dirs):
    all_labels = []
    for annotation_dir in annotation_dirs:
        for annotation_file in os.listdir(annotation_dir):
            if annotation_file.endswith('.json'):
                annotation_path = os.path.join(annotation_dir, annotation_file)
                with open(annotation_path, 'r') as f:
                    annotation = json.load(f)
                img_labels = [obj["classTitle"] for obj in annotation.get('objects', [])]
                all_labels.extend(img_labels)  # Note: extending, not appending
    return set(all_labels)  # Returning as a set to ensure uniqueness

class MyDataset(Dataset):
    def __init__(self, img_dir, ann_dir, transform=None):
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.img_list = [img for img in os.listdir(img_dir) if img.endswith((".jpg", ".jpeg", ".png"))]
        self.transform = transform
        self.all_labels = all_labels

        resize_transform = transforms.Resize((224, 224))

        self.imgs = []
        for index in range(len(self.img_list)):
            img_path = os.path.join(self.img_dir, self.img_list[index])
            img = cv.imread(img_path)
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = resize_transform(img)
            self.imgs.append(img)

        self.labels = []
        for index in range(len(self.img_list)):
            annotation_path = os.path.join(self.ann_dir, self.img_list[index] + '.json')
            with open(annotation_path, 'r') as f:
                annotation = json.load(f)
                img_labels = [obj["classTitle"] for obj in annotation.get('objects', [])]
            binary_labels = [1 if label in img_labels else 0 for label in self.all_labels]
            self.labels.append(binary_labels)
        print("Finish dataload in mydataset")

    def __getitem__(self, index):
        # img_path = os.path.join(self.img_dir, self.img_list[index])
        # img = cv.imread(img_path)
        # img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        # img = Image.fromarray(img)

        # label_path = os.path.join(self.ann_dir, self.img_list[index] + '.json')
        # with open(label_path, 'r') as f:
        #     annotation = json.load(f)
        #     img_labels = [obj["classTitle"] for obj in annotation.get('objects', [])]
        # binary_labels = [1 if label in img_labels else 0 for label in self.all_labels]

        img = self.imgs[index]
        if self.transform:
            img = self.transform(img)
        # return img, torch.tensor(binary_labels).float()
        return img, torch.tensor(self.labels[index]).float()

    def __len__(self):
        return len(self.img_list)


# data preprocess
transform = transforms.Compose([
    transforms.RandomRotation(15),  # 随机旋转15度
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),  # 随机调整亮度、对比度和饱和度
    transforms.Resize((224, 224)),  # 调整图像尺寸以匹配模型输入
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


train_img_dir = '/content/dataset/food-recognition-2022-DatasetNinja/training/img'
train_annotation_dir = '/content/dataset/food-recognition-2022-DatasetNinja/training/ann'
validation_img_dir = '/content/dataset/food-recognition-2022-DatasetNinja/validation/img'
validation_annotation_dir = '/content/dataset/food-recognition-2022-DatasetNinja/validation/ann'
batch_size = 32
num_classes=498
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


all_labels = collect_all_labels([train_annotation_dir, validation_annotation_dir])
train_dataset = MyDataset(train_img_dir, train_annotation_dir, transform=transform)
validation_dataset = MyDataset(validation_img_dir, validation_annotation_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)

Finish dataload in mydataset
Finish dataload in mydataset


In [14]:
#b0
model = EfficientNet.from_pretrained('efficientnet-b0').to(device)

num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, num_classes).to(device)

criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1, verbose=True)

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    best_val_loss = float('inf')  # Start with the worst case; the lowest loss is best
    best_model_path = '/content/drive/MyDrive/models/best_model.pth'  # Define path to save the best model


    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        print(f"Training Epoch {epoch+1}/{num_epochs}")

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

        val_loss = evaluate_model(model, criterion, test_loader)
        scheduler.step(val_loss)

        # Save the model if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_model(model, best_model_path)
            print(f"New best model saved with validation loss: {val_loss:.4f}")


def evaluate_model(model, criterion, data_loader):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            predicted = outputs.softmax(1)
            predicted[predicted >= 0.1] = 1
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    val_loss = running_loss / len(data_loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}")
    return val_loss

def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved successfully at {path}")

train_model(model, criterion, optimizer, scheduler, num_epochs=10)



Finish dataload in mydataset
Finish dataload in mydataset
Loaded pretrained weights for efficientnet-b0
Training Epoch 1/10


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/10], Loss: 0.0202
Validation Loss: 0.0162, Accuracy: 0.4730
Model saved successfully at /content/drive/MyDrive/models/best_model.pth
New best model saved with validation loss: 0.0162
Training Epoch 2/10
Epoch [2/10], Loss: 0.0137
Validation Loss: 0.0146, Accuracy: 0.5520
Model saved successfully at /content/drive/MyDrive/models/best_model.pth
New best model saved with validation loss: 0.0146
Training Epoch 3/10
Epoch [3/10], Loss: 0.0125
Validation Loss: 0.0140, Accuracy: 0.6040
Model saved successfully at /content/drive/MyDrive/models/best_model.pth
New best model saved with validation loss: 0.0140
Training Epoch 4/10
Epoch [4/10], Loss: 0.0118
Validation Loss: 0.0141, Accuracy: 0.6310
Training Epoch 5/10
Epoch [5/10], Loss: 0.0111
Validation Loss: 0.0137, Accuracy: 0.6280
Model saved successfully at /content/drive/MyDrive/models/best_model.pth
New best model saved with validation loss: 0.0137
Training Epoch 6/10
Epoch [6/10], Loss: 0.0105
Validation Loss: 0.0137, Accuracy: 0

In [ ]:
def predict_first_ten_images(model, data_loader, label_list):
    model.eval()  # Set the model to evaluation mode
    images, actual_labels = next(iter(data_loader))  # Get the first batch of images
    images = images.to(device)

    with torch.no_grad():  # No need to track gradients for predictions
        outputs = model(images)
        outputs = torch.sigmoid(outputs)  # Apply sigmoid to convert outputs to probabilities

    # Thresholding predictions for binary classification
    predictions = outputs > 0.5

    # Decode predictions to labels
    decoded_predictions = []
    decoded_actual_labels = []
    for i in range(predictions.shape[0]):
        predicted_labels = [label_list[idx] for idx, val in enumerate(predictions[i]) if val == 1]
        actual_labels_list = [label_list[idx] for idx, val in enumerate(actual_labels[i]) if val == 1]
        decoded_predictions.append(predicted_labels)
        decoded_actual_labels.append(actual_labels_list)

    return images.cpu(), decoded_predictions, decoded_actual_labels

# Convert all_labels from set to sorted list if it's not already a list
if isinstance(all_labels, set):
    all_labels = sorted(list(all_labels))

# Assuming the train_loader has a batch size of at least 10 and shuffle=False for consistent results
images, predicted_labels, actual_labels = predict_first_ten_images(model, train_loader, all_labels)

# Now you can print the results or process them further
for idx in range(10):
    print(f"Image {idx+1}:")
    print("Predicted Labels:", ', '.join(predicted_labels[idx]))
    print("Actual Labels:", ', '.join(actual_labels[idx]))
    # Add image display logic if required

In [19]:
import csv

def save_labels_to_csv(labels, file_path):
    """ Saves a list of labels to a CSV file. """
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        for label in labels:
            writer.writerow([label])  # Each label on a new row

# Convert all_labels from set to sorted list if it's not already a list
if isinstance(all_labels, set):
    all_labels = sorted(list(all_labels))

# Define the path where you want to save the CSV
csv_file_path = 'labels_list.csv'

# Save labels to CSV
save_labels_to_csv(all_labels, csv_file_path)
print(f"Labels have been saved to {csv_file_path}.")


Labels have been saved to labels_list.csv.


In [ ]:
import csv

def load_label_calories(csv_file_path):
    label_calories = {}
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            label, calories = row[0], float(row[1])
            label_calories[label] = calories
    return label_calories

def predict_and_calculate_calories(model, data_loader, label_list, label_calories):
    model.eval()  # Set the model to evaluation mode
    images, actual_labels = next(iter(data_loader))  # Get the first batch of images
    images = images.to(device)

    with torch.no_grad():  # No need to track gradients for predictions
        outputs = model(images)
        outputs = torch.sigmoid(outputs)  # Apply sigmoid to convert outputs to probabilities

    # Thresholding predictions for binary classification
    predictions = outputs > 0.5
    decoded_predictions, decoded_actual_labels = [], []
    predicted_calories, actual_calories = [], []

    for i in range(predictions.shape[0]):
        predicted_labels = [label_list[idx] for idx, val in enumerate(predictions[i]) if val == 1]
        actual_labels_list = [label_list[idx] for idx, val in enumerate(actual_labels[i]) if val == 1]

        decoded_predictions.append(predicted_labels)
        decoded_actual_labels.append(actual_labels_list)

        # Calculate calories
        predicted_calories.append(sum(label_calories.get(label, 0) for label in predicted_labels))
        actual_calories.append(sum(label_calories.get(label, 0) for label in actual_labels_list))

    return images.cpu(), decoded_predictions, decoded_actual_labels, predicted_calories, actual_calories

# Usage
label_calories = load_label_calories('labels_list.csv')
images, predicted_labels, actual_labels, predicted_calories, actual_calories = predict_and_calculate_calories(model, train_loader, all_labels, label_calories)

# Output results
for idx in range(10):
    print(f"Image {idx+1}:")
    print("Predicted Labels:", ', '.join(predicted_labels[idx]))
    print("Actual Labels:", ', '.join(actual_labels[idx]))
    print(f"Predicted Calories: {predicted_calories[idx]}, Actual Calories: {actual_calories[idx]}")

In [ ]:
# @title
import matplotlib.pyplot as plt

def plot_calories_with_lines(predicted_calories, actual_calories):
    # Number of images
    n_images = len(predicted_calories)

    # Create a range for the number of images
    index = range(1, n_images + 1)

    # Set up the plot
    plt.figure(figsize=(10, 5))

    # Plot lines for actual and predicted calories
    plt.plot(index, actual_calories, marker='o', linestyle='-', color='b', label='Actual Calories')
    plt.plot(index, predicted_calories, marker='x', linestyle='--', color='r', label='Predicted Calories')

    # Adding Xticks and labels
    plt.xlabel('Image Index')
    plt.ylabel('Calories')
    plt.title('Comparison of Actual and Predicted Calories')
    plt.xticks(index)  # Ensure each image index is marked

    # Adding legend
    plt.legend(loc='upper right')

    # Show the plot
    plt.grid(True)  # Adding a grid for better readability
    plt.show()

# Call the function with your data
plot_calories_with_lines(predicted_calories, actual_calories)